<a href="https://colab.research.google.com/github/AAyyoouubb/TP_ML/blob/master/Project_Regression_fv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries,
* **numpy** for matrix structures
* **pandas** for reading tables
* **matplotlib**  for visualization


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix as sm
import random
%matplotlib inline
import warnings  
warnings.filterwarnings('ignore')

### **First Part:**



Pandas function `read_csv()` is used to read the csv file **data8.csv** and place it as a dataframe.

In [2]:
data =pd.read_csv("drive/MyDrive/project_machinglearning_S3/data8.csv")
data= data.sample(frac=1)

FileNotFoundError: ignored

Split the dataset into inputs(x) and output(y). Use the method values to transform from a DataFrame object to an array object, which can efficiently managed by Numpy library.

In [ ]:
y=data.y.values.reshape(-1,1)

x=data["x"].values.reshape(-1,1)


Add a bias column to the input vector. This is a column of ones so when we calibrate the parameters it will also multiply such bias.

In [ ]:
X = np.concatenate((np.ones((len(x),1)),x),axis=1)

Spliting the data into **80%** training set and **20%** testing set.

In [ ]:
trainingSplit = int(.8 * data.shape[0])

x_train = x[:trainingSplit]
x_test = x[trainingSplit:]

X_train = X[:trainingSplit]
X_test = X[trainingSplit:]

y_train = y[:trainingSplit] 
y_test = y[trainingSplit:]



There are one feature x and one column y



In [ ]:
print(*data.columns)

In [ ]:
print(x_train.shape,y_train.shape)
print(x_test.shape,X_test.shape)

No invalid value was found in the data

In [ ]:
pd.isna(data).any()


The describtion of the data: 

feature| mean | median | min | max
--- | --- | --- | --- | ---
x | 11.45|-0.1 |0|22.9
y | 25.94| 157.93|-0.43|50.44

In [ ]:
print(data.describe())


Ploting the training set 

In [ ]:
plt.scatter(x_train, y_train)

Create a function `grad()` to compute the necessary gradients of the cost function. 


In [ ]:
m = data.shape[0]

# calculate gradient
def grad(theta,X,y):
    dJ = 1/m*np.sum((X.dot(theta)-y)*X,axis=0).reshape(-1,1)
    return dJ

Similarly, calculate the cost function `cost()`, also known as objective function, which can be expressed as the sum of the squared errors, as follows

In [ ]:
def cost(theta,X,y):
    J = np.sum((X.dot(theta)-y)**2,axis=0)[0]/m
    return J



Armijo function for linear search the learning rate

In [ ]:
def armijo(X,y,theta,grad=grad,cost=cost):
    eps = .001
    eta = 10
    alpha =10e-15    
    gradient = grad(theta,X,y)
    phiprzero = -gradient.T.dot(gradient)
    phizero = cost(theta,X,y)
    while  cost(theta-alpha*gradient,X,y)<= alpha * phiprzero * eps + phizero :
      alpha*=eta
    return alpha/eta


We are ready to implement the Gradient Descent algorithm
The parametres :
* `X` features;
* `y` labels;
* `learning_rate` learning rate for parametres update;
* `epochs` max iteration of gradient descent update;
* `TOL` is a tolerance, i.e a maximum difference between the values of the parameters between iterations so it can be stated that the values converged;
* `use_armijo` True to opt the armijo rule to calculate the learning rate otherwise the algorithm will use the fixed learning rate.


The steps of this algorithm consists of:
* Obtain the gradients of the cost function according the actual value of the parameters;
* Calculate the cost to keep track of it;
* Update the parameters;

In [ ]:
def GD(X,y,learning_rate = 0.001,epochs=100,TOL=1e-7,use_armijo = False,verbose=False):
    theta = np.array([[0]]*X.shape[1])
    if verbose:print(theta)
    theta_history = [theta]
    J_history = [cost(theta,X,y)]
    
    thetanew = theta
    if verbose:print(f'epoch \t Cost(J,X,y) \t')
    for epoch in range(epochs):
        if epoch%100 == 0:
            if verbose:print(f'{epoch:5d}\t{J_history[-1]:7.4f}\t')
        dJ = grad(theta,X,y)
        J = cost(theta,X,y)
        if use_armijo:learning_rate= armijo(X,y,theta)
        thetanew = theta - learning_rate*dJ
        theta_history.append(thetanew)
        J_history.append(J)
        
        if np.sum((thetanew - theta)**2) < TOL:
            if verbose:print('Convergence achieved.')
            break
        theta = thetanew
    if verbose:print(theta)
    return thetanew,theta_history,J_history

Create and train the 4 models ( learning rate = 0.001,0.002,0.003 and using armijo rule)

In [ ]:
models = []
models.append( GD(X_train,y_train,learning_rate = 0.001))
models.append( GD(X_train,y_train,learning_rate = 0.002))
models.append( GD(X_train,y_train,learning_rate = 0.003))
models.append( GD(X_train,y_train,use_armijo=1))

Drawing the fit line for the 4 models;

In [ ]:
plt.scatter(x_train, y_train)

colors = ['red' , 'blue', "green","black"]
labels = ['0.001','0.002','0.003','Armijo']
for label,color,model in zip(labels,colors,models):
  theta = model[0]
  Y_pred = X_train.dot(theta)
  plt.plot([min(x_train), max(x_train)], [min(Y_pred), max(Y_pred)], color=color,label = label) # predicted
plt.legend()
plt.show()

In [ ]:
colors = ['red' , 'blue', "green","black"]
labels = ['0.001','0.002','0.003','Armijo']
for label,color,model in zip(labels,colors,models):
  J_history = model[2]
  plt.plot(J_history, color=color,label = label) # predicted
plt.legend()
plt.show()

Generalization error  

In [ ]:
for model in models:
  theta =model[0]
  print(cost(theta,X_test,y_test))

R²  score or the coefficient of determination explains how much the total variance of the dependent variable can be reduced by using the least square regression.

In [ ]:
def accuracy(y_actual,y_pred):
  # sum of square of residuals
  ssr = np.sum((y_pred - y_actual)**2)
  #  total sum of squares
  sst = np.sum((y_actual - np.mean(y_actual))**2)
  # R2 score
  r2_score = 1 - (ssr/sst)
  return r2_score


The accuracy of the 4 models

In [ ]:
for model in models:
  theta =model[0]
  print(accuracy(y,X.dot(theta)))

In [ ]:
print(X.shape)

# Bias and variance

In [ ]:
def bootstrap(samples, X,y):
  X_train_sets = []
  y_train_sets = []

  ix = int(0.7*X.shape[0])
  for sample in range(samples):
    ix = np.random.randint(X.shape[0],size=X.shape[0])

    xx= X[ix,:] 
    yy= y[ix,:]

    X_train_sets.append(xx)
    y_train_sets.append(yy)

  return X_train_sets,y_train_sets

X_train_sets,y_train_sets = bootstrap(10,X_train,y_train)

In [ ]:
def bias_var(X_train_sets,y_train_sets,X_test,y_test):
  samples = len(X_train_sets)
  predicts = np.zeros((samples,y_test.shape[0],1))
  for ix in range(samples):
    theta,theta_history,J_history = GD(X_train_sets[ix],y_train_sets[ix],use_armijo=1)
    predicts[ix,:] =X_test.dot(theta)

  mean_preds = np.mean(predicts)
  bias_per_point = (mean_preds-y_test)**2
  bias = np.mean(bias_per_point)
  var_per_point = np.mean((predicts-mean_preds)**2,axis=0)
  var = np.mean(var_per_point)
  return bias,var


In [ ]:
bias,var = bias_var(X_train_sets,y_train_sets,X_test,y_test)
print("bias = ",bias," variance =",var)

### **Second Part**

In [ ]:
data = pd.read_csv("drive/MyDrive/project_machinglearning_S3/data4.csv")

data= data.sample(frac=1)

print(data.shape)

Spliting to traing and testing

In [ ]:
y=data.y.values.reshape(-1,1)
x=data["x"].values.reshape(-1,1)

print(x.shape,y.shape)
trainingSplit = int(.8 * data.shape[0])


x_train = x[:trainingSplit]
y_train = y[:trainingSplit] 
x_test = x[trainingSplit:]
y_test = y[trainingSplit:]

 The data contain 2 features `x` and `y`.

In [ ]:
print(*data.columns)

The size of training set is **192**, the size of testing size is **48**.









In [ ]:
print(x_train.shape,y_train.shape)
print(x_test.shape,X_test.shape)


The describtion of our dataset :

 feature| mean | median | min | max
--- | --- | --- | --- | ---
x | -0.1|-0.1 |-24|23.8
y | 193.31| 157.39|-67.13|578.49

In [ ]:
data.describe()

Using `matplotlib.scatter` to plot our **2D** dataset as scatters.

In [ ]:
plt.scatter(x_train, y_train)


Create the tranformation `tf` which take the initial features and the degree and give the new features according to pylinomial transformation of given degree.

we will use the same gradient descent implimentated in part 1.

In [ ]:
def tf(x,deg):
  X=np.ones((len(x),1))
  for i in range(1,deg+1):
    X = np.concatenate((X,x**i),axis=1)
  return X

Apply the tranformation for degree = 2

In [ ]:
k=2
X_train =tf(x_train,deg=k)
X_test =tf(x_test,deg=k)
X =tf(x,deg=k)
print(X.shape)

**Q4.** Fiting the model;

In [ ]:

theta,theta_history,J_history = GD(X_train,y_train,use_armijo=1)
print("Error generalization =",cost(theta,X_test,X_test))

Draw result

In [ ]:
plt.scatter(x_train, y_train)
Y_pred = X_train.dot(theta)
plt.scatter(x_train, Y_pred, color="red") # predicted
plt.title("degree = "+str(k))
plt.show()

In [ ]:
y_pred =X_test.dot(theta)
print("Accurcy = ",accuracy(X_test,y_pred))

In [ ]:
plt.plot(J_history)


Bias varience estimation for the 4 models:

In [ ]:
for d in [1,2,4]:
  X_train = tf(x_train,deg = d)
  X_test = tf(x_test,deg = d)
  X_train_sets,y_train_sets = bootstrap(10,X_train,y_train)
  bias,var = bias_var(X_train_sets,y_train_sets,X_test,y_test)
  print("deg ",d,"bias",bias,"var",var)

### PART3

To apply the logistic regression we need to normalise

In [ ]:
ynorm = y.copy()
maxy = np.max(y)
miny = np.min(y)
ynorm = (y-miny)/(maxy - miny) 


Create the function for the logistic model

In [ ]:
def sigmoid(z): return 1 / (1 + np.e**(-z))
m = data.shape[0]

# calculate gradient
def grad2(theta,X,y):
    y_hat = sigmoid(np.dot(X, theta))
    dJ = 1/m*np.sum((y_hat-y)*X,axis=0).reshape(-1,1)
    return dJ
def cost2(theta,X, y):         
    z = np.dot(X, theta)
    predict_1 = y * np.log(sigmoid(z))
    predict_0 = (1 - y) * np.log(1 - sigmoid(z))
    return -sum(predict_1 + predict_0) / len(X)

def LR(X,y,learning_rate = 0.001,epochs=1000,TOL=1e-7,use_armijo = False):
    theta = np.array([[0]]*X.shape[1])
    print(theta)
    theta_history = [theta]
    J_history = [cost2(theta,X,y)]
    
    thetanew = theta
    print(f'epoch \t Cost2(J,X,y) \t')
    for epoch in range(epochs):
        if epoch%100 == 0:
            print(f'{epoch:5d}\t{J_history[-1]}\t')
        dJ = grad2(theta,X,y)
        J = cost2(theta,X,y)
        if use_armijo:learning_rate= armijo(X,y,theta,grad2,cost2)
        thetanew = theta - learning_rate*dJ
        theta_history.append(thetanew)
        J_history.append(J)
        
        if np.sum((thetanew - theta)**2) < TOL:
            print('Convergence achieved.')
            break
        theta = thetanew

    return thetanew,theta_history,J_history

fit the model

In [ ]:
theta,theta_history,J_history = LR(X,ynorm,use_armijo=1)


Plot the diserid in blue, the predicted in red

In [ ]:
plt.scatter(x, ynorm)
Y_pred = sigmoid( X.dot(theta))
plt.scatter(x,Y_pred, color="red") 
plt.show()

The losse **history**

In [ ]:
plt.plot(J_history)